
# CT421 Project 1  Evolutionary Search - GAs

**Aoife Mulligan 20307646 | Leo Chui 20343266**  



# Part B



## Representation

- Individuals are lists of binary strings
- Each string represents a bin, and all strings are of length $n$, the number of items
- 0 means that the item is not in current bin, and 1 means the item is in current bin
- length of list would be Number of bins
- also have an array which stores weights
  - e.g. for bin: \[10*1*01\], we can get the italicised 1's weight by getting the weight at the 2nd index in the weights array
  
### Recommended Testing bits

- compare repair performance with non-repair counterpart

### Miscellaneous bits

- Less heuristics could be a way to look at it

### Aoife's Notes

Mutation swapping seems like the only way to do mutation - as in, the nature of the problem means we can't mutate without swapping.
E.G. if n = 4 and two of our bins are: \[1100\] and \[0011\], we want to mutate only one chromosome. The first chromosome gets selected, so we flip the bit. Then we would have \[0100\] and \[0011\], but now the first item is not in a bin. We either must create an extra bin to place the first item (which seems crazy but I guess we could try), or we flip the corresponding bit in bin 2, so that the item is now in the other bin.

The only difference between this basic example and our implementation is that our implementation will have many bins, and so when we want to swap a bit, we should randomly select the bin for it to be swapped to.




# Part B



#### Read Problem File


In [74]:
# read problem file to get items and weights

# open binpacking.txt
with open("Binpacking.txt", "r") as file:
    lines = file.readlines()

# removes till 'BPP      1'
for _ in range(17):
    lines.pop(0)

# from here, text file repeats in pattern
# BPP      i
# number of different weights
# capacity of bin
# weight          number of items (repeats number of different weights times)

CAPACITIES = []
ITEM_WEIGHTS = []

# loop through 5 BPP scenarios
for i in range(5):
    # remove seperator BPP i
    lines.pop(0)

    # saves number of different weights
    num_weights = int(lines.pop(0))
    # saves capacity of bin
    CAPACITIES.append(int(lines.pop(0)))
    # make new map for BPP scenario i
    ITEM_WEIGHTS.append([]) # append empty list
    
    # loop for weights number of times
    for j in range(num_weights):
        # split line into weight and number of items with weight
        line = lines.pop(0).split()

        # for each item of weight W, append W to weights[i] n times
        # n being the number of items with weight W, at line[1]
        for k in range(int(line[1])):
            ITEM_WEIGHTS[i].append(int(line[0]))
            print("Appending item with weight", line[0], "to BPP", i+1)
# close file
file.close()

# print for sanity check
"""
for i in range(5):
    print("BPP", i+1)
    print(ITEM_WEIGHTS[i])
    print(CAPACITIES[i])
"""

Appending item with weight 200 to BPP 1
Appending item with weight 200 to BPP 1
Appending item with weight 200 to BPP 1
Appending item with weight 199 to BPP 1
Appending item with weight 198 to BPP 1
Appending item with weight 198 to BPP 1
Appending item with weight 197 to BPP 1
Appending item with weight 197 to BPP 1
Appending item with weight 194 to BPP 1
Appending item with weight 194 to BPP 1
Appending item with weight 193 to BPP 1
Appending item with weight 192 to BPP 1
Appending item with weight 191 to BPP 1
Appending item with weight 191 to BPP 1
Appending item with weight 191 to BPP 1
Appending item with weight 190 to BPP 1
Appending item with weight 190 to BPP 1
Appending item with weight 189 to BPP 1
Appending item with weight 188 to BPP 1
Appending item with weight 188 to BPP 1
Appending item with weight 187 to BPP 1
Appending item with weight 187 to BPP 1
Appending item with weight 186 to BPP 1
Appending item with weight 185 to BPP 1
Appending item with weight 185 to BPP 1


'\nfor i in range(5):\n    print("BPP", i+1)\n    print(ITEM_WEIGHTS[i])\n    print(CAPACITIES[i])\n'

In [75]:
import matplotlib.pyplot as plt
import random
import math

CROSSOVER_RATE = 0.9
POPULATION_SIZE = 100
NUM_GENERATIONS = 1000
ELITE_FACTOR = 0.1



### Helper functions


In [76]:
# helper functions

# calculate bin weights

def bin_sum_weight(bin, item_weight):
    # debug print to keep track of where we are
    print("Calculating bin weight")
    sum_weight = 0
    for i in range(len(bin)):
        if int(bin[i]) == 1:
            sum_weight += item_weight
    print("Sum weight is ", sum_weight)
    return sum_weight

# check clashes between bins
def check_clashes(bins):
    # XOR all bins should be a string of all 1s
    # return true if no clash
    for i in range(len(bins[0])):
        count_ones = sum([int(bins[j][i]) for j in range(len(bins))])
        if count_ones > 1:
            return False
    print("No clash")
    return True


### Fitness Function and optimum bin amount calc


In [77]:
# bin-packing fitness function
# calculates fitness by determining error for each bin
# actual bin weight - capacity
# perfect bin = 0

def capacity_error_fitness(bin, item_weights, capacity=1000):
    sum_weight = bin_sum_weight(bin, item_weights)
    error = (sum_weight - capacity) ** 2
    return error

In [78]:
# Tournament selection function

# selects k individuals from the population at random
# returns the index of the best individual

def tournament_selection(population, scores, k=3):
    # Select k individuals from population at random
    selection_i = random.randrange(len(population))
    for i in [random.randrange(len(population)) for _ in range(k-1)]:
        # Check if better (e.g. perform a tournament)
        if scores[i] < scores[selection_i]:
            selection_i = i
    # Return the index of the best
    return population[selection_i]


In [79]:
# mutation function
# each item has a 1/n chance of being mutated
# if mutated, item is swapped with another item

def mutate(bins, mutation_rate=None):
    if mutation_rate is None:
        mutation_rate = 1/float(len(bins))
    # choose a random bin from bins
    bin1 = random.choice(bins)
    # choose a random item from the bin
    for i in range(len(bin1)):
        if random.random() < mutation_rate:
            # swap item in the bin with a corresponding item from another bin
            # flip the current bit in the string
            bin2 = random.choice(bins)
            # first check bin1 != bin2
            # second check that bin2[i] does not equal bin1[i]
            if bin1 != bin2 and bin2[i] != bin1[i]:
                # swap the item with a corresponding item from another bin
                bin1 = bin1[:i] + str(int(bin1[i]) ^ 1) + bin1[i+1:]
                bin2 = bin2[:i] + str(int(bin2[i]) ^ 1) + bin2[i+1:]
    return bins

In [80]:
# TODO: crossover function


In [81]:
# elitism function

def elite_select(population, scores, elite_factor=ELITE_FACTOR):
    elite_size = int(len(population) * elite_factor)
    # Sort the population based on scores in ascending order
    sorted_population = [x for _, x in sorted(zip(scores, population))]

    # Select the elite individuals
    elite = sorted_population[:elite_size]

    # Return the elite individuals
    return elite


In [82]:
# BPP population generation function

def generate_population(item_weights, bpp_scenario, population_size=POPULATION_SIZE):
    print("Population size is", population_size)
    population = []
    # while population is not full
    while len(population) < population_size:
        individual = []
        
        # make copy of item_weights at bpp_scenario
        item_list = item_weights[bpp_scenario].copy()

        # make a zero string as initial bin
        bin = ''.join('0' for _ in range(len(item_list)))
        
        # for each item in item_weights, check if weight would violate capacity
        for i in range(len(item_list)):
            # if weight does not violate capacity, flip 1 to individual
            if bin_sum_weight(bin, item_list[i]) + item_list[i] < CAPACITIES[0]:
                bin = bin[:i] + '1' + bin[i+1:]
                print("Adding item ", item_list[i], " to bin ", bin)
            else:
                # if weight violates capacity, append bin to individual
                print("Weight ", item_list[i], " violates capacity")
                individual.append(bin)
                print("Bin ", bin, " full, appended to individual")
                # here, it does not go back to create a new bin
                # attempt to fix: ::pray::
                bin = ''.join('0' for _ in range(len(item_list)))
                # print("New bin created")
        population.append(individual)
        print("Individual appended to population")
        print("Population is now", population)
        print("\n-------------------\n")
        """
        # make sure no strings in individual contains the same item
        # and append to population if no clash
        if check_clashes(individual):
                    population.append(individual)
        """
        return population


### Algorithm


In [83]:

# change algorithm so that crossover is only performed if the function is passed in

# Generic GA algorithm that is passed different functions

def algorithm(item_weights, pop_size=POPULATION_SIZE,
              elite_factor=ELITE_FACTOR, fitness=capacity_error_fitness, cross_rate=CROSSOVER_RATE,
              crossover=None):
    print(pop_size)
    print(item_weights)
    # define average fitness list to plot
    avg_fitness = []

    # initial random population
    population = generate_population(item_weights)
    # debug - print size of population
    print("Population size: %d" % len(population))
    print("Population: ", len(population))
    # debug - print length of first bin of first solution
    # print("Individual length: %d" % len(population[0][0]))
    # debug - print first solution
    # print("First solution: %s" % population[0])

    # Store temp best solution
    best_solution, best_fitness = 0, fitness(population[0][0], item_weights)

    # Loop generations
    for gen in range(NUM_GENERATIONS):
        # Evaluate all individuals in population
        scores = []
        for individual in population:
            # individual_fitness is the sum of the fitness of each bin of that individual
            individual_fitness = 0
            for bin in individual:
                individual_fitness += fitness(bin, item_weights)
            # store individual fitness in scores
            scores.append(individual_fitness)

        # calculate average fitness of generation
        avg_fitness.append(sum(scores) / pop_size)

        # print best score every 100 generations
        if gen % 100 == 0:
            print("Generation %d, best score = %.3f, average fitness = %.3f" %
                  (gen, min(scores), avg_fitness[gen]))

        # Check for new best solution
        for i in range(pop_size):
            # print(scores[i], population[i])
            if scores[i] < best_fitness:
                best_solution, best_fitness = population[i], scores[i]
                print("Generation %d, new best f(%s) = %.3f" %
                      (gen, population[i], scores[i]))

        # Select parents to mutate or crossover
        parents = [tournament_selection(population, scores)
                   for _ in range(pop_size)]
        # Create next generation
        children = list()
        children.extend(elite_select(population, scores, elite_factor))

        for i in range(0, pop_size):
            children.append(mutate(parents[i]))
        # Replace population
        population = children

    # plot average fitness over generations
    plt.plot(avg_fitness)
    plt.xlabel('Generation')
    plt.ylabel('Average Fitness')
    plt.show()

    return [best_solution, best_fitness]

In [84]:
if __name__ == '__main__':
    print("BPP - 1")
    print("Running algorithm...")
    # TODO: 
    # solution, score = algorithm(ITEM_WEIGHTS)
    population = generate_population(ITEM_WEIGHTS, 0)
    
    
    print("Completed operations")

BPP - 1
Running algorithm...
Population size is 100
Calculating bin weight
Sum weight is  0
Adding item  200  to bin  1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Calculating bin weight
Sum weight is  200
Adding item  200  to bin  1100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Calculating bin weight
Sum weight is  400
Adding item  200  to bin  1110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Calculating bin weight
Sum weight is  597
Adding item  199  to bin  1111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Calculating bin weight
Sum weight is  792
Adding item  198  to bin  1111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Calculating bin weight
Sum weight is  990
Weight  198  violates capacity
Bin  111110000000000000000000000